# LangChain: Evaluation
Outline:
- Example generation
- Manual evaluation (and debuging)
- LLM-assisted evaluation
- LangChain evaluation platform

## Create our QandA application

In [1]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [2]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [3]:
from models import get_embeddings_model

embedding = get_embeddings_model()

In [4]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch, embedding=embedding
).from_loaders([loader])

/opt/anaconda3/envs/ml/lib/python3.13/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [5]:
from models import get_chat_model

llm = get_chat_model(temperature=0.0)

In [6]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

### Coming up with test datapoints

In [7]:
print(data[10])
print(data[11])

page_content=': 10
name: Cozy Comfort Pullover Set, Stripe
description: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.

Size & Fit
- Pants are Favorite Fit: Sits lower on the waist.
- Relaxed Fit: Our most generous fit sits farthest from the body.

Fabric & Care
- In the softest blend of 63% polyester, 35% rayon and 2% spandex.

Additional Features
- Relaxed fit top with raglan sleeves and rounded hem.
- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.

Imported.' metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 10}
page_content=': 11
name: Ultra-Lofty 850 Stretch Down Hooded Jacket
description: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly 

### Hard-coded examples

In [8]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### LLM-Generated examples

In [9]:
from langchain.evaluation.qa import QAGenerateChain
example_gen_chain = QAGenerateChain.from_llm(llm)

In [10]:
example_gen_chain

QAGenerateChain(verbose=False, prompt=PromptTemplate(input_variables=['doc'], input_types={}, partial_variables={}, template='You are a teacher coming up with questions to ask on a quiz. \nGiven the following document, please generate a question and answer based on that document.\n\nExample Format:\n<Begin Document>\n...\n<End Document>\nQUESTION: question here\nANSWER: answer here\n\nThese questions should be detailed and be based explicitly on information in the document. Begin!\n\n<Begin Document>\n{doc}\n<End Document>'), llm=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-exp', google_api_key=SecretStr('**********'), temperature=0.0, top_p=0.9, top_k=1, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x30ea4b250>, default_metadata=()), llm_kwargs={})

In [12]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/opt/anaconda3/envs/ml/lib/python3.13/site-packages/langchain/chains/llm.py:370: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [13]:
new_examples[0]

{'qa_pairs': {'query': "According to the document, what type of innersole is used in the Women's Campside Oxfords and what feature does it include?",
  'answer': "The Women's Campside Oxfords use a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control."}}

In [14]:
data[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

### Combine examples

In [15]:
mapped_examples = list(map(lambda x: x['qa_pairs'], new_examples))
mapped_examples

[{'query': "According to the document, what type of innersole is used in the Women's Campside Oxfords and what feature does it include?",
  'answer': "The Women's Campside Oxfords use a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control."},
 {'query': 'What are the dimensions of the small and medium Recycled Waterhog Dog Mats, Chevron Weave?',
  'answer': 'The small mat\'s dimensions are 18" x 28" and the medium mat\'s dimensions are 22.5" x 34.5".'},
 {'query': "What percentage of the sun's harmful rays does the Infant and Toddler Girls' Coastal Chill Swimsuit block?",
  'answer': "The Infant and Toddler Girls' Coastal Chill Swimsuit blocks 98% of the sun's harmful rays."},
 {'query': 'What materials are used in the body of the Refresh Swimwear, V-Neck Tankini Contrasts and what percentage of each material is used?',
  'answer': 'The body of the Refresh Swimwear, V-Neck Tankini Contrasts is made of 82% recycled nylon and 18% Lycra® spandex.'},
 {'query': 'Accord

In [16]:
examples += mapped_examples
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "According to the document, what type of innersole is used in the Women's Campside Oxfords and what feature does it include?",
  'answer': "The Women's Campside Oxfords use a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control."},
 {'query': 'What are the dimensions of the small and medium Recycled Waterhog Dog Mats, Chevron Weave?',
  'answer': 'The small mat\'s dimensions are 18" x 28" and the medium mat\'s dimensions are 22.5" x 34.5".'},
 {'query': "What percentage of the sun's harmful rays does the Infant and Toddler Girls' Coastal Chill Swimsuit block?",
  'answer': "The Infant and Toddler Girls' Coastal Chill Swimsuit blocks 98% of the sun's harmful rays."},
 {'query': 'What materials are used in the body of the Refresh Swim

In [17]:
qa.run(examples[0]["query"]) # invoke()

/var/folders/l4/vd99d1jj46l4pg5l2_852z280000gn/T/ipykernel_69802/1491569966.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(examples[0]["query"]) # invoke()




> Entering new RetrievalQA chain...

> Finished chain.


'Yes, the Cozy Comfort Pullover Set has side pockets.'

## Manual Evaluation

In [18]:
import langchain
langchain.debug = True

In [19]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 73\nname: Cozy Cuddles Knit Pullover Set\ndescription: Perfect for lounging, this knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out. \n\nSize & Fit \nPants are Favorite Fit: Sits lower on the waist. \nRelaxed Fit: Our most generous fit sits farthest from the body. \n\nFabric & Care \nIn the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features \nRelaxed fit top with raglan sleeves and rounded hem. \nPull-on pants have 

'Yes, the Cozy Comfort Pullover Set has side pockets.'

In [20]:
# Turn off the debug mode
langchain.debug = False

## LLM assisted evaluation

In [21]:
predictions = qa.batch(examples) # batch()



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [22]:
from langchain.evaluation.qa import QAEvalChain

In [24]:
eval_chain = QAEvalChain.from_llm(llm)

In [26]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [28]:
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'}]

In [30]:
graded_outputs[0]

{'results': 'CORRECT'}

In [29]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set has side pockets.
Predicted Grade: CORRECT

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.
Predicted Grade: CORRECT

Example 2:
Question: According to the document, what type of innersole is used in the Women's Campside Oxfords and what feature does it include?
Real Answer: The Women's Campside Oxfords use a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control.
Predicted Answer: According to the document, the Women's Campside Oxfords use a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control.
Predicted Grade: CORRECT

Example 3:
Question: What are the dimensions of the small and medium Recycled Waterhog Dog Mats, 